**Завдання:**

Використовуючи клас **GaussianNB** з бібліотеки **Scikit-Learn** створіть модель класифікації для навчання з використанням датасету **Iris** (sklearn.datasets.load_iris). Після цього, за допомогою методів з бібліотеки **ml_metrics**, обрахуйте метрики для отриманої моделі.

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.datasets import load_iris

df = load_iris()
X_train, X_test, y_train, y_test = train_test_split(df.data, df.target, random_state=42)

In [40]:
gm = GaussianNB().fit(X_train, y_train)

In [41]:
test_predict = gm.predict(X_test)

In [42]:
print(test_predict)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


In [43]:
print(y_test)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


In [44]:
print(f"accuracy : {metrics.accuracy_score(y_test, test_predict)}")

accuracy : 1.0


In [45]:
import ml_metrics
ml_metrics.mse(y_test, test_predict)

0.0